In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import numpy as np
import itertools
import dpa_load
import multiprocessing as mp
from datetime import datetime
from itertools import repeat
import split_articles
import numeric_articles
import continue_articles
# we import some functions from the Handelsblatt folder
import sys
sys.path.insert(1, os.getcwd().replace('dpa code', 'Handelsblatt'))
import count_words_mp
import identify_eng
import correct_url
import clean_dpa_articles
import clean_dpa_references

In [2]:
# Set the number of cores to use
NUM_CORE = mp.cpu_count()-4

# DPA Data (1991 - 2018)

Deutsche PresseAgentur (DPA) is the Germany's biggest news agency which sells its news reports to the leading German newspapers. We believe that the data set has a high chance to be useful for economic forecasting because DPA produces information that is timely and has a large reach.

We purchased DPA data in November 2019. The corpus consists of **7,539,874** articles from January 1991 to December 2018.

The data set includes news from both dpa-Basisdienstes and dpa-afx Wirtschaftsnachrichten. The former one is the basic news service covering such topics as Economy, Politics, and  Finance. The second one was created in 1999. It specializes in financial news.

## Load the data

First, we read in the data by extracting the following XML elements:

* title - article's title
* text - text of the article
* date - publication date
* ressort - section (Politics vs Economy)
* source/credit - source (dpa vs afx)
* city - which city the news article refers to
* genre - journalistic genre, e.g., chronology, story, table
* wortanzahl - word count
* keywords - keywords associated with an article

In [3]:
# Folder with unpacked articles
#path = r'E:\\Userhome\\jbaer\\dpa_unpacked'

#path = r'G:\\Test\\Results\\dpa Raw Data\\dpa_unpacked'
path = os.getcwd().replace('\\newspaper_data_processing\\dpa code', '') + '\\dpa_unpacked'

folder_list = []

# 2 folders for dpa and dpa-afx 
for fol in [fol for fol in os.listdir(path)]:

    # Within each folder: folders for different years
    for f in [f for f in os.listdir(path + '\\' + fol)]:
        folder_list.append(path + '\\' + fol + '\\' + f)

In [4]:
# Select a path to the folder for storing results
#PATH = r'G:\\Test\\Results'
#os.chdir(PATH)

In [5]:
# Use the 'dpa_load' function to load articles
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    df_list = pool.map(dpa_load.dpa_load, folder_list)
    data = pd.concat(df_list)
    data.reset_index(inplace=True, drop=True)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:45:16.705110


In [6]:
print(len(data))

7539874


In [7]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [8]:
data.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,5739189.xml,1,1,1991,pl,dpa,Schalck-Golodkowski,Schalck: Milliardenkredit sicherte Zahlungsfäh...,Berlin,,218,WiPo
1,Tschads Regierung: Bevölkerung soll Waffen abl...,5739191.xml,1,1,1991,pl,dpa,Tschad,Tschads Regierung: Bevölkerung soll Waffen abl...,N'Djamena,,75,WiPo
2,Welajati: Iran bleibt bei einem Krieg am Golf ...,5739193.xml,1,1,1991,pl,dpa,Golfkrise Iran,Welajati: Iran bleibt bei einem Krieg am Golf ...,Teheran,,90,WiPo
3,Bush will offenbar seinen Außenminister erneut...,5739195.xml,1,1,1991,pl,dpa,Golfkrise USA,Bush will offenbar seinen Außenminister erneut...,Washington,,181,WiPo
4,Morgenzusammenfassung Neue Runde diplomatische...,5739199.xml,1,1,1991,pl,dpa,Golfkrise,Morgenzusammenfassung Neue Runde diplomatische...,Washington/Luxemburg,,504,WiPo


In [9]:
#data.to_csv('dpa_raw.csv')

In [3]:
data = pd.read_csv('dpa_raw.csv', encoding = 'utf-8', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'})

In [4]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [5]:
data.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,5739189.xml,1,1,1991,pl,dpa,Schalck-Golodkowski,Schalck: Milliardenkredit sicherte Zahlungsfäh...,Berlin,,218,WiPo
1,Tschads Regierung: Bevölkerung soll Waffen abl...,5739191.xml,1,1,1991,pl,dpa,Tschad,Tschads Regierung: Bevölkerung soll Waffen abl...,N'Djamena,,75,WiPo
2,Welajati: Iran bleibt bei einem Krieg am Golf ...,5739193.xml,1,1,1991,pl,dpa,Golfkrise Iran,Welajati: Iran bleibt bei einem Krieg am Golf ...,Teheran,,90,WiPo
3,Bush will offenbar seinen Außenminister erneut...,5739195.xml,1,1,1991,pl,dpa,Golfkrise USA,Bush will offenbar seinen Außenminister erneut...,Washington,,181,WiPo
4,Morgenzusammenfassung Neue Runde diplomatische...,5739199.xml,1,1,1991,pl,dpa,Golfkrise,Morgenzusammenfassung Neue Runde diplomatische...,Washington/Luxemburg,,504,WiPo


# Pre-processing

## Light pre-processing

### Remove short articles (<100 words)

Short articles are often incoherent or contain only insiginicant news. For this reason we decided to filter out articles that consist of less than 100 words. 

In [6]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text.replace('PARAGRAPH', ' ') for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:01:59.225678


In [7]:
# Save the result as a new column "word_count"
data['word_count'] = count_results

In [8]:
# remove articles with less than 100 words
data = data[data['word_count']>=100]
data.reset_index(inplace=True, drop=True)
print(len(data))

5364634


In [9]:
#data.to_csv('dpa_prepro_step1.csv')

### Remove exact duplicates

A few examples of duplicates in our corpus:
* The same article enters the corpus twice with different publication dates (e.g., 10.1.1991 and 11.1.1991). In this case, a natural solution is to keep the first entry.
* The same article appears twice with a slight variation in the metadata (e.g., the word count is a little different even though the articles are identical, or the keywords differ).
* The same article enters the corpus twice with the same publication date and metadata.
* The same article is published by dpa and dpa-afx (in this case, we keep an article published by dpa).

In [10]:
data = data.sort_values(['year', 'month', 'day', 'topic'], ascending=[True, True, True, True]) # dpa articles come before afx articles
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [11]:
# All the duplicated articles are saved as 'dpa_duplicates' for further exploration.
dpa_duplicates = data[data['texts'].duplicated(keep = False)]
len(dpa_duplicates)

1595777

In [12]:
dpa_duplicates.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic,word_count
13,Neues Schema für Investmentkurse Achtung: Vom ...,5739254.xml,1,1,1991,wi,dpa,KURSE DREI A),Neues Schema für Investmentkurse Achtung: Vom ...,,,544,WiPo,283
15,NEUES INVESTMENTSCHEMA. 125) FF RESERVE FONDS ...,5739268.xml,1,1,1991,wi,dpa,KURSE DREI B,NEUES INVESTMENTSCHEMA.,,,489,WiPo,258
16,NEUES INVESTMENTSCHEMA. 247) RE-INRENTA 248) R...,5739273.xml,1,1,1991,wi,dpa,KURSE DREI C),NEUES INVESTMENTSCHEMA.,,,430,WiPo,262
209,Neues Schema für Investmentkurse Achtung: Vom ...,5740026.xml,2,1,1991,wi,dpa,KURSE DREI A),Neues Schema für Investmentkurse Achtung: Vom ...,,,544,WiPo,283
210,NEUES INVESTMENTSCHEMA. 125) FF RESERVE FONDS ...,5740028.xml,2,1,1991,wi,dpa,KURSE DREI B,NEUES INVESTMENTSCHEMA.,,,489,WiPo,258


Some articles are more than once in the corpus. We filter out all duplicates and only keep the articles with the oldest date or the articles published by dpa.

In [13]:
data.drop_duplicates(['texts'], keep = 'first', inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4542316


In [14]:
#data.to_csv('dpa_prepro_step2.csv')

### Corrections and updates of text news

There are two types of articles that we consider separately: text news corrections and updates.

* In text news corrections (Berichtigung or Berichtigte Neufassung), journalists usually change only a few facts. More rarely, they add or rewrite several paragraphs. In most of the cases, corrected and original news texts are pubished on the same day.

* In the updated texts news (Aktualisierung), journalists add a small amount of text to reflect the latest status. These articles can be published a few days later than the original article. 

#### Corrected news texts

The corrected news texts do not contain much new information, so we treat them as duplicates. However, before deleting these articles, we check if we can always find the original articles in the dataset.

Often the titles of corrected news articles include the titles of the original articles. For example, the title of the corrected article 'Berichtigung: Zahl der betroffenen US-Staaten Clinton will 23,5 Millionen Hektar Nationalforst schützen.' includes information on what was changed (Berichtigung: Zahl der betroffenen US-Staaten) and the title of the original article 'Clinton will 23,5 Millionen Hektar Nationalforst schützen.'.

We use this fact and try to find pairs of original and corrected news reports with the same title. While the titles of corrected news reports may have a different format, being able to find pairs of articles sharing the same title in each time period means that there are no temporal changes in the data.  

Corrected news reports contain 'Berichtigung' (correction) or 'Berichtigte Neufassung' (corrected version) in the title. However, we exclude the articles on rectified shares that include 'Berichtigungsaktien' in the title.

In [15]:
# Corrected news reports in dpa data.
Berichtigung = data[(data.title.str.contains('Berichtigung|Berichtigte Neufassung')) & (~data.title.str.contains('Berichtigungsaktien')) & (data.topic == 'WiPo')]

In [16]:
# track time
startTime = datetime.now()
# List with a publication year of the corrected articles
inputs_year = []
# List with a publication month of the corrected articles
inputs_month = []
# List with a publication day of the corrected articles
inputs_day = []
# List with the titles of the corrected articles
inputs_Berichtigung_title = []
# List with the titles of all the articles published on the same day as the corrected article
inputs_titles = []
# In this project we will concentrate on dpa rather than dpa-afx data. Please see explanation below.
dpa_data = data[data.topic=='WiPo']

for ind in Berichtigung.index:
    inputs_year.append(Berichtigung['year'][ind])
    inputs_month.append(Berichtigung['month'][ind])
    inputs_day.append(Berichtigung['day'][ind])
    inputs_Berichtigung_title.append(Berichtigung['title'][ind])
    inputs_titles.append(list(dpa_data[(dpa_data['year'] == Berichtigung['year'][ind]) & (dpa_data['month'] == Berichtigung['month'][ind]) & (dpa_data['day'] == Berichtigung['day'][ind])]["title"]))        
inputs = list(zip(inputs_year, inputs_month, inputs_day, inputs_Berichtigung_title, inputs_titles))
print(datetime.now()-startTime)

0:13:15.663261


We use the function `Berichtigung_pairs` to output the dataframe that contains the titles of the corrected and original articles along with their publication date. This dataframe will help us understand if there are some temporal changes in the data.

In [17]:
startTime = datetime.now()

import Berichtigung_pairs
import pandas as pd

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    Berichtigung_intermediate = pool.map(Berichtigung_pairs.Berichtigung_pairs, inputs)
    Berichtigung_df = pd.concat(Berichtigung_intermediate) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:26.443538


We can see that this dataframe includes 16982 titles of the corrected and original articles, while the total number of corrected articles is equal to 34221.

In [18]:
Berichtigung_df = Berichtigung_df.reset_index()
print("The number of pairs found = {}".format(len(Berichtigung_df)))
print("The number of corrected articles = {}".format(len(Berichtigung)))
Berichtigung_df['day'] = list(map(int, Berichtigung_df.day))
Berichtigung_df['month'] = list(map(int, Berichtigung_df.month))
Berichtigung_df['year'] = list(map(int, Berichtigung_df.year))
Berichtigung_df.head()

The number of pairs found = 16892
The number of corrected articles = 34221


,index,day,month,year,title_Berichtigung,title_main
0,0,7,1,1991,Berichtigte Neufassung dpa 159 - DLW - Bietigh...,DLW legte 1990 zu - besonders Möbelbereich erf...
1,0,22,1,1991,Berichtigte Neufassung - Wiesbaden/1257 Preise...,Preise in den neuen Ländern bei starken Bewegu...
2,0,7,2,1991,Berichtigte Neufassung amnesty - London/0107 )...,amnesty international setzt sich für US-Wehrdi...
3,0,7,2,1991,Berichtigte Neufassung - Bundeshaushalt - Bonn...,Bundeshaushalt 1991 mit 400 Milliarden Mark fa...
4,0,8,4,1991,Berichtigte Neufassung - München/1321 Patentam...,Patentamt erwartet Schub durch ostdeutsche Erf...


To understand why this is the case, we create a dataframe with the number of corrected articles (column 'texts') and identified pairs (column 'title_Berichtigung') per year.

We can see that before 1999, the number of corrected articles is low, which explains why the number of identified original articles is also low. Between 1999 and 2011, the number of corrected articles ranges between between 900 and 2000, and we successfully identify most of the original articles that have the same title as their corrections.

However, from 2012 onwards, the number of original articles that we are able to find decreases significantly. After further exploration, we realised that over this period of time most of the original articles had been removed. Therefore, we decided to delete only those corrected articles that were published before 2012 to avoid the risk of getting rid of unique articles.

In [19]:
df1 = Berichtigung.groupby('year').nunique()['texts'].to_frame().reset_index()
df2 = Berichtigung_df.groupby('year').nunique()['title_Berichtigung'].to_frame().reset_index()
merged_df = df1.merge(df2, how = 'left')
merged_df['title_Berichtigung'] = merged_df['title_Berichtigung'].astype('Int64')
merged_df

,year,texts,title_Berichtigung
0,1991,103,16
1,1992,138,22
2,1993,148,20
3,1994,136,26
4,1995,123,36
5,1996,164,45
6,1997,177,56
7,1998,428,182
8,1999,1323,678
9,2000,1245,673


Here we delete the corrected articles published before 2012.

In [20]:
data.drop(data[(data['title'].str.contains('Berichtigung|Berichtigte Neufassung', na = False)) & (~data['title'].str.contains('Berichtigungsaktien', na=False)) & \
               (data.topic == 'WiPo') & (data.year < 2012)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4519997


#### Updated news texts

The updated news texts might contain relatively large chunks of new information. Therefore, we do not want to delete them. Instead, we have done the following:

1. If we manage to find an updated news text and its original version and both are published on the same day, we delete the original article as it does not contain the latest information.

2. If the update and the original article are published on different days, we keep both news reports. The original article is important because it was published on the day market participants received the news. At the same time, the updated artilce might contain important new information. If there are several updates, we only keep the latest one as containing the full information. In case the latest update is very close to the original article, it will be removed later by the `fuzzy_duplicates` function.

In [21]:
from aktualisierung import delete_aktualisierung_index
# In this project we will concentrate on dpa rather than dpa-afx data. Please see explanation below.
# We do not consider articles that contain 'Nachrichtenüberblick' in the keywords because these articles 
# contain multiple articles.
dpa_data = data[(data.topic=='WiPo') & (~data.keywords.str.contains('Nachrichtenüberblick'))]
dates = dpa_data.groupby(['year', 'month', 'day'])
dates = list(dates.groups)
dates = np.array_split(dates, NUM_CORE)
chunks = [pd.concat([dpa_data[(dpa_data['year'] == t[0]) & (dpa_data['month'] == t[1]) & (dpa_data['day'] == t[2])] for t in tup]) for tup in dates]
# Use the 'delete_aktualisierung_index' function to get a list with indices of all indermediate updates and 
# original articles published on the same day as the respective updates.
startTime = datetime.now()
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    aktualisierung_index = pool.map(delete_aktualisierung_index, chunks)
    pool.close()
    pool.join()
print(datetime.now()-startTime)
aktualisierung_index = [idx for l in aktualisierung_index for idx in l]

0:03:47.488227


In [22]:
# Delete indermediate updates and original articles published on the same day as the respective updates based on indices.
data.drop(aktualisierung_index, inplace = True)
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True)
print(len(data))

4518993


## Duplicates specific to article types

After exploring corrected and updated news texts, we realised that there are other types of articles that suffer from the problem of duplication. These are summaries (Zusammenfassung, Gesamtzusammenfassung, Morgenfassung), overviews (Überblick), repeated articles (Wiederholung), and advance notifications (Vorausmeldung).

* __Summaries (Zusammenfassung, Gesamtzusammenfassung, Morgenfassung)__

Often we may see two versions of the same (or almost the same) article on the same day: the first is the original article and the second is a summary of that article. Many summaries are actually similar to updated articles because they consist of the original article and a new piece of text. However, in some cases, the summary is a condensed version of the original article including only the most important information. In any case, the content of the summary and the original article are very similar or identical. Therefore, if we can identify an original article and a summary published on the same day and with the same title, we consider the shorter article as a duplicate. In our corpus, we have 49,773 duplicates of this type.

* __Overviews (Überblick)__

In most cases, overviews are stand-alone articles containing all the information about a particular event available on a particular day. Sometimes, however, journalists write an article about a certain event in the morning, and then publish an overview in the evening, which is an extended version of the original article describing the latest developments. These overivews are very similar to updated news texts. We find 4,675 overviews with the same title as the original articles and published on the same day. We remove the short article from the overview and the original article.

* __Repeated articles (Wiederholung)__

We found 1,482 repeated articles with the same title as the original articles and published on the same day. These articles are either identical to the original articles or contain a small correction ('Berichtigte Wiederholung'). We remove the short article from the repeated article and the original article.

* __Advance notifications (Vorausmeldung)__

Advance notifications are news articles about events that will take place in the future. We found 606 advance notifications that have the same title as other articles published on the same day. In these cases, the advance notifications are a shorter version of the original articles. If we can identify an original article and an advance notification published on the same day and with the same title, we consider the shorter article as a duplicate.

First, we use the function `duplicates_pairs` to output the data frame with the titles of duplicates and original articles along with their publication day.

In [23]:
# Select article types that potentially suffer from the duplication problem. These are summaries (Zusammenfassung, 
# Gesamtzusammenfassung, Morgenfassung), overviews (Überblick), repeated articles (Wiederholung), and advance notifications
# (Vorausmeldung).

# In this project we will concentrate on dpa rather than dpa-afx data. Please see explanation below.

# We do not consider articles that contain 'dpa-Vorausmeldungen kompakt' in the title or 'Nachrichtenüberblick' in the 
# keywords because these articles contain multiple articles.

dup_types = data[(data.title.str.contains('Zusammenfassung|zusammenfassung|Überblick|Wiederholung|Vorausmeldung')) & (data.topic == 'WiPo') & \
                 (~data.title.str.contains('dpa-Vorausmeldungen kompakt')) & (~data.keywords.str.contains('Nachrichtenüberblick'))]
len(dup_types)

# Dates on which at least one summary, overview, repeated article, or advance notification is published.
dates = dup_types.groupby(['year', 'month', 'day'])
dates = list(dates.groups)
# Exclude dpa-afx data and articles that contain 'Nachrichtenüberblick' in the keywords.
dpa_data = data[(data.topic=='WiPo') & (~data.keywords.str.contains('Nachrichtenüberblick'))]
# Create a list of data frames, where each data frame contains all the articles published on the same day as one of 
# the summaries, overviews, repeated articles, or advance notifications.
day_df = [dpa_data[(dpa_data['year'] == t[0]) & (dpa_data['month'] == t[1]) & (dpa_data['day'] == t[2])] for t in dates]

In [24]:
startTime = datetime.now()
# Use the function duplicate_pairs to return the dataframe with the titles of duplicates and original articles 
# along with their publication date.
import duplicates_pairs

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    duplicates_intermediate = pool.map(duplicates_pairs.duplicates_pairs, day_df)
    duplicates_df = pd.concat(duplicates_intermediate) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:56.124459


Out of 705,975 summaries, overviews, repeated articles, and advance notifications, we find 56,535 articles that we consider to be duplicates.

In [25]:
duplicates_df = duplicates_df.reset_index()
print("The number of pairs found = {}".format(len(duplicates_df)))
print("The number of summaries, overviews, repeated articles, or advance notifications = {}".format(len(dup_types)))
duplicates_df['day'] = list(map(int, duplicates_df.day))
duplicates_df['month'] = list(map(int, duplicates_df.month))
duplicates_df['year'] = list(map(int, duplicates_df.year))
duplicates_df.head()

The number of pairs found = 56536
The number of summaries, overviews, repeated articles, or advance notifications = 705975


,index,day,month,year,title_duplicate,title_original
0,0,2,1,1991,Zusammenfassung Ernst von Siemens gestorben.,Ernst von Siemens gestorben.
1,0,5,1,1991,Zusammenfassung Drei Polizisten in Bologna ers...,Drei Polizisten in Bologna erschossen.
2,0,9,1,1991,Morgenzusammenfassung Baker und Asis in Genf -...,Bush fordert vom Kongreß Zustimmung zu Militär...
3,0,10,1,1991,Vorausmeldung Gesamtberliner Parlament konnsti...,Gesamtberliner Parlament konnstituiert sich.
4,0,14,1,1991,Zweite Zusammenfassung Gorbatschow rechtfertig...,Gorbatschow rechtfertigt Armee-Einsatz in Lita...


We use the function `delete_duplicates` to output indices of the duplicate articles. We then delete the articles with these indices.

In [26]:
startTime = datetime.now()
# Use the function delete_duplicates to output indices of the duplicate articles.
import delete_duplicates

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    duplicates_index = pool.map(delete_duplicates.delete_duplicates, day_df)
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)
duplicates_index = [idx for l in duplicates_index for idx in l]

0:01:13.606906


In [27]:
# Delete duplicate articles.
data.drop(duplicates_index, inplace = True)
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True)
print(len(data))

4461990


In [28]:
#data.to_csv('dpa_prepro_step3.csv')

## Filtering

Unfortunately, dpa articles are not as consistently sorted into sections and subsections as articles from other 
news media. Instead, we investigate the most commonly used titles and keywords and remove irrelevant articles based on them.

We exclude irrelevant articles based on the following titles

* 1) Londoner Edelmetallpreise: Precious metal prices (without text)
* 2) Tageskalender: List of upcoming events
* 3) (Tabelle), TABELLE: : Tables in text form
* 4) SPORT, Sport (except for the titles that contain TRANSPORT, INTERSPORT, PASSPORT, Sportartikel, news about sportswear manufacturers, sporting goods industry, SPORTARTIKLER, Sportmodelle, or Sportswear): news related to Sports. Beware that news about sports marketing agencies (e.g., Sportfive), sports media websites (e.g., Sportal), and new sports models of car manufacturers might be removed as well. 
* 5) KORREKTUR: Article corrections
* 6) Impressum: Dpa contact data
* 7) Testmeldung: Test-articles from dpa
* 8) Kurse A, Kurse B, Kurse C, Kurse D, KURSE A, KURSE B, KURSE C, KURSE DREI, KURSE Drei, Kurse drei: Stock charts without text
* 9) DGAP-DD: DGAP reports
* 10) New Yorker Aktien-Schlußkurse: Stock closing prices at the New York stock exchange (articles only occur from 1997 to 2002)
* 11) VERMISCHTES: Miscellaneous with no relation to economics
* 12) Angekündigte US-Quartalszahlen auf einen Blick: Quarterly US figures
* 13) Terminvorschau: Appointment preview
* 14) Aus der Landespolitik: News reports on regional politics. We think they are unlikely to be important for our research question. 
* 15) Die Woche in Berlin, Die Woche in Bonn, Die politische Woche: Announcements of political (and economic) news for next week. The format is very different from other news reports and difficult for the models we use.
* 16) Notizen aus der Politik, NOTIZEN AUS DER POLITIK: Collection of varios short political articles which contentwise  do not seem to be relevant for our research question and are covered only within the limited time period, 2002-2011.
* 17) Presseschau (except for the titles that contain 'zu'): Press reviews that include only headlines of important news reports from various media outlets. We can not analyze the full texts and headlines together because different types of data require different models. Articles with headlines containing both 'Presseschau' and 'zu' are actual press reviews (not just headlines) of a single newspaper on a single topic.
* 18) ÜBERBLICK: Analysten-Umstufungen: Changes in stock ratings (different article format).
* 19) Chronologie, CHRONOLOGIE: Chronology - important dates and events. We remove these articles because for our research question backward-looking articles are arguable not that important.
* 20) dpa-Landesdienst: Regional news unlikely to be important for our research question.
* 21) Die Top-Themen am Aktienmarkt: Short news articles about stock market. They do contain relevant information, but they also have a very different format and they were first issued in 2011. 
* 22) Aktien Asien Schluss.: Quantitative information about the stock market.
* 23) Börsentag auf einen Blick: Articles about the stock market, different format, a lot of quantitative information.
* 24) US-Quartalszahlen vom Vortag: Quarterly stock market figures, tables.
* 25) Achtung - Sonderdisposition: A schedule of the upcoming news articles on a particular topic.
* 26) dpa-Vorausmeldungen kompakt: Multiple short news articles on the upcoming events. The length of each of the articles usually does not exceed 100 words.

In [29]:
# Filter out non-economic articles based on titles.
fil_titles = '''Londoner Edelmetallpreise|Tageskalender|\(Tabelle\)|SPORT|Sport|KORREKTUR|Impressum|Testmeldung|Kurse A[^a-z]|Kurse B[^a-z]|Kurse C[^a-z]|Kurse D[^a-z]|KURSE A|KURSE B|KURSE C|KURSE DREI|Kurse drei|KURSE Drei|Kurse/drei|DGAP-DD|New Yorker Aktien-Schlusskurse|VERMISCHTES|Angekündigte US-Quartalszahlen|Terminvorschau|Aus der Landespolitik|Die Woche in Berlin|Die Woche in Bonn|Die politische Woche|Notizen aus der Politik|NOTIZEN AUS DER POLITIK|ÜBERBLICK: Analysten-Umstufungen|TABELLE:|Chronologie|CHRONOLOGIE|dpa-Landesdienst|Die Top-Themen am Aktienmarkt|Aktien Asien Schluss\.|Börsentag auf einen Blick|US-Quartalszahlen vom Vortag|Achtung - Sonderdisposition|dpa-Vorausmeldungen kompakt'''
titles_exc = '''TRANSPORT|INTERSPORT|PASSPORT|Sportartikel|SPORTARTIKLER|Sportmodelle|Sportswear'''
data.drop(data[(data['title'].str.contains(fil_titles, na = False)) & (~data['title'].str.contains(titles_exc, na=False))].index, inplace=True)
# Filter out press reviews that consist of headlines only
data.drop(data[(data['title'].str.contains('Presseschau', na = False)) & (~data['title'].str.contains('zu', na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4091041


We exclude articles based on a title and a text:

* 1) A title contains 'Überblick: ANALYSTEN-EINSTUFUNGEN', and a text contains 'Folgende Investmentbanken haben sich': Changes in stock ratings (different article format).

In [30]:
# Filter out articles based on a title and a text.
data.drop(data[data.title.str.contains('Überblick: ANALYSTEN-EINSTUFUNGEN', na = False) & data.texts.str.contains('Folgende Investmentbanken haben sich', na=False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4088048


We exclude articles based on a title and keywords:

* 1) A title contains 'Dispositionen', and keywords contain 'Wahl' or 'wahl': quantitative information about elections (different article format).

In [31]:
# Filter out articles based on a title and keywords.
data.drop(data[data.title.str.contains('Dispositionen', na = False) & data.keywords.str.contains('Wahl|wahl', na=False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4087623


We exclude irrelevant articles based on the following sections.

* 1) Tabelle: Tables in text form (some articles are still left after the previos step)
* 2) Historisches: News about historical events
* 3) Achtung: Announcemt of upcoming news

In [32]:
# Filter out non-economic articles based on sections.
fil_genres = '''Tabelle|Historisches|Achtung'''
data.drop(data[data['genre'].str.contains(fil_genres, na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4066855


We exclude non-economic articles based on the following keywords.
* 1) Redaktionshinweis: Editor's notes for Dpa journalists
* 2) DGAP: DGAP reports
* 3) Sport, SPORT, SPO (except for Sportartikel, this section contains articles on sports companies): Sport news (some sports articles are still left after the previos steps)
* 4) Kurse A, Kurse B, Kurse C, Kurse D, KURSE A, KURSE B, KURSE C, Kurse D,
     KURSE DREI, Kurse drei, KURSE Drei, KURSE drei, Kurse Drei, Kurse/drei: Stock charts without text (some articles are  still left after the previos steps)
* 5) Tagesvorschau, Vorschau, VORSCHAU, vorschau: List of titles of upcoming news
* 6) Bilderdienst: Dpa Picture Service
* 7) Geschichte: News related to historical events
* 8) Landespolitik: Regional news irrelevant for economic forecasting
* 9) dpa-Morgenlage: News about what happened last night. These articles have a special format: quick and very short reports.

In [33]:
# Filter out non-economic articles based on keywords.
fil_keywords = '''Redaktionshinweis|DGAP|Sport|SPORT|SPO|Kurse A|Kurse B|Kurse C|Kurse D|KURSE A|KURSE B|KURSE C|KURSE DREI|Kurse drei|KURSE Drei|KURSE drei|Kurse Drei|Kurse/drei|Kurse D|Tagesvorschau|Vorschau|vorschau|VORSCHAU|Bilderdienst|Geschichte|Landespolitik|dpa-Morgenlage'''
keywords_exc = '''Sportartikel'''
data.drop(data[(data['keywords'].str.contains(fil_keywords, na = False)) & (~data['keywords'].str.contains(keywords_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3826563


We exclude articles based on the following bits of text.
* 1) Schalterverkaufskurse: Precios metal prices
* 2) dpa-news.de: News regarding the Dpa website
* 3) Wirtschafts- und Finanztermine, Wirtschafts- und Finanz-Termine, Konjunktur- und Wirtschaftstermine: List of dates when economic data will be published/economic events will take place
* 4) DGAP (except for articles that contain DGAP standing for Deutsche Gesellschaft für Auswärtige Politik): DGAP reports
* 5) Bitte verwenden Sie diese Meldung nicht: Retracted articles
* 6) [§] 26 Abs., § 15a WpHG 1, § 15 WpHG, Artikel 19 MAR, article 19 Market Abuse Regulation (MAR): Regulatory news
* 7) Die Pivotpunkte für den Dax-Future: Pivot points for the Dax-Future
* 8) An der Frankfurter Wertpapierbörse wurden, Die Aktien im Dow Jones EuroStoxx 50, Die Aktien im Dow Jones Euro Stoxx 50: Stock charts
* 9) Ihr Ansprechpartner: Redaktion Politik International: List of current political news headlines
* 10) (Achtung - Sonderdisposition): A schedule of the upcoming news articles on a particular topic

In [34]:
fill_text = '''Schalterverkaufskurse:|dpa-news\.de|Wirtschafts- und Finanztermine|Wirtschafts- und Finanz-Termine|DGAP|Bitte verwenden Sie diese Meldung nicht|Konjunktur- und Wirtschaftstermine|[§] 26 Abs\. 1|§ 15a WpHG|§ 15 WpHG|Artikel 19 MAR|article 19 Market Abuse Regulation \(MAR\)|Die Pivotpunkte für den Dax-Future|An der Frankfurter Wertpapierbörse wurden|Die Aktien im Dow Jones EuroStoxx 50|Die Aktien im Dow Jones Euro Stoxx 50|\(Achtung - Sonderdisposition\)'''
text_exc = '''Auswärtige Politik'''
data.drop(data[(data['texts'].str.contains(fill_text, na = False)) & (~data['texts'].str.contains(text_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3693403


We want to exclude irrelevant articles based on the following two sources.
* 1) dpa-frei: Article corrections
* 2) dpa-wahl: Articles about federal election results

In [35]:
data.drop(data[data['source'].str.contains('dpa-frei|dpa-wahl', na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3693373


We exclude articles regarding dpa itself.

In [36]:
data.drop(data[data['city'] == 'Die Deutsche Presse-Agentur'].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3693368


We keep articles on two subjects: Politics ('wi') and Economy ('wi'). Articles with the 'rs' value of the column 'rubrics' are removed, because 'rs' stands for editorial management. 

In [37]:
data = data[(data['rubrics']==u'wi') | (data['rubrics']==u'pl')]
data.reset_index(inplace=True, drop=True)
print(len(data))

3692200


Remove four articles published in 2019. 

In [38]:
data = data[data['year']<2019]
data.reset_index(inplace=True, drop=True)
print(len(data))

3692196


In [39]:
#data.to_csv('dpa_prepro_step4.csv')

### Split up articles

Sometimes multiple articles are collected and merged into one entry. For example, articles with the title, keyword, or genre 
'Nachrichtenüberblick' are a collection of the most important articles of the day. Because these smaller articles
can have different sentiments and topics, we separate articles that consist of multiple smaller articles. Articles consisting of multiple smaller articles can be identified with the following words which can appear in titles, keywords, or genres.

- dpa-Nachrichtenüberblick, Nachrichtenüberblick: An overview of news for the upcomming days or news which are a few days old
- Kurznachrichten Wirtschaft: Collection of short economic news
- Analysten-Einstufungen, ANALYSTEN-EINSTUFUNGEN: Analyst stock ratings

We delete one of the articles containing 'Nachrichtenüberblick' in the title because it has too many errors that make it difficult to split the article into paragraphs.

In [40]:
# The article to delete
file_drop = ['8180362.xml']
ind = data[data.file == '8180362.xml']['texts'].index[0]
data[data.file == '8180362.xml']['texts'][ind]

' PARAGRAPH     Rentenreform verabschiedet\n  PARAGRAPH     BERLIN - Der Weg für die Förderung der privaten Altersvorsorge\nmit knapp 21 Mrd. DM ist frei.  e K\n B h e i r t de. Auch das rot-rot-regierte Mecklenburg-Vorpommern\nstimmte zu. Die SPD-CDU-Koalition von Bremen enthielt sich. In\nMecklenburg-Vorpommern droht nach der überraschenden Zustimmung von\nMinisterpräsident Ringstorff eine Koalitionskrise. Nach PDS-Ansicht\nhat die SPD den Koalitionsvertrag gebrochen und damit den\nRegierungspartner herausgefordert.\n  PARAGRAPH    Mieter erhalten mehr Rechte\n  PARAGRAPH    BERLIN - Die mehr als 20 Mio. Mieter in Deutschland erhalten von\nSeptember an mehr Rechte. Die Grenze für Mieterhöhungen wird gesenkt\nund die Kündigungsfristen werden zu Gunsten der Mieter geändert. Das\nsieht die Mietrechtsreform der Bundesregierung vor, die am Freitag im\nBundesrat die letzte parlamentarische Hürde nahm. Bisher betrugen die\nKündigungsfristen für Mieter un d Vermieter gleichermaßen maximal\nz

In [41]:
data.drop(data[data['file'].isin(file_drop)].index, inplace=True)

While testing the code, we have decided to delete a quantitative part of the following two articles:

In [42]:
ind1 = data[data.file == '8833807.xml']['texts'].index[0]
data[data.file == '8833807.xml']['texts'][ind1]

' PARAGRAPH     BERLIN(dpa-AFX) - Mit dem Bundeshaushalt 2004 und dem Finanzplan\nbis 2007 bleibt die von der Bundesregierung geplante Neuverschuldung\nvorerst im kritischen Bereich. Sie soll jetzt im kommenden Jahr 28,8\nMilliarden Euro betragen, nachdem Finanzminister Hans Eichel (SPD)\nund die Koalition übereingekommen sind, 2 Milliarden Erlöse aus dem\nVerkauf weiterer Telekom- <DTE.ETR> und Postaktien <DPW.ETR> an die\nKreditanstalt für Wiederaufbau zu erzielen. Diese soll das Paket\nspäter bei guten Kursen an der Börse platzieren.\nFür das laufende Jahr hat Eichel inzwischen eine Neuverschuldung\nvon 35 Milliarden in den Haushaltsentwurf eingestellt, spricht aber\nbereits von einer Verdoppelung der zunächst gesetzlich\nverabschiedeten 18,9 Milliarden Euro. Dies wären etwa 38 Milliarden\nneue Schulden. Die im Herbst erwartete Höhe soll dann in einen\nNachtragshaushalt aufgenommen werden. Andere Fachleute der Koalition\nsprechen schon von rund 40 Milliarden Euro. Die Opposition nen

In [43]:
ind2 = data[data.file == '8833809.xml']['texts'].index[0]
data[data.file == '8833809.xml']['texts'][ind2]

' PARAGRAPH     BERLIN (dpa-AFX) - Der Entwurf des Bundeshaushalts 2004 sieht\nAusgaben in Höhe von 251,2 Milliarden Euro vor. Das sind 1,2 Prozent\nmehr als für das laufende Jahr gesetzlich geplant, aber 2,3 Prozent\nweniger als das Finanzministerium jetzt für 2003 mit 257 Milliarden\nEuro erwartet. Die neuen Schulden sind mit 28,8 Milliarden Euro\neingeplant.\nDas größte Ausgabevolumen verwaltet das Ressort für Gesundheit und\nSoziales. Die hohe Steigerungsrate im Arbeits- und Wirtschaftsetat\nvon Minister Wolfgang Clement (SPD) wird durch den hohen Zuschuss an\ndie Bundesanstalt für Arbeit von 5,2 Milliarden Euro bestimmt.\n\nDie Übersicht über die Einzelpläne:\nSollEntwurfVeränderung\n20032004zum Vorjahr\n(in Millionen Euro)(Prozent)\n--------------------------------------\nBundespräsidialamt20,4722,96+ 12,2\nBundestag540,73548,93+1,5\nBundesrat17,0617,80+4,3\nBundeskanzleramt1.483,561.488,06+0,3\nAuswärtiges Amt2.229,912.183,40-2,1\nInnen4.014,004.092,58+2,0\nJustiz345,35344,27-0,

In [44]:
data.at[ind1, 'texts'] = data['texts'][ind1].split(' Hier die Eckwerte')[0]
data.at[ind2, 'texts'] = data['texts'][ind2].split('\n\nDie Übersicht')[0]

In [45]:
s_mult_art = '''dpa-Nachrichtenüberblick|Nachrichtenüberblick|Kurznachrichten Wirtschaft|Analysten-Einstufungen|ANALYSTEN-EINSTUFUNGEN'''
mult_art = data[data['title'].str.contains(s_mult_art, na = False)]
mult_art = mult_art.append(data[data['keywords'].str.contains(s_mult_art, na = False)])
mult_art = mult_art.append(data[data['genre'].str.contains(s_mult_art, na = False)])
mult_art.drop_duplicates(['texts'], keep = 'first', inplace=True)
mult_art.reset_index(inplace=True, drop=True)

In [46]:
# delete 'mult_art' from the original data
data.drop(data[data['title'].str.contains(s_mult_art, na = False)].index, inplace=True)
data.drop(data[data['keywords'].str.contains(s_mult_art, na = False)].index, inplace=True)
data.drop(data[data['genre'].str.contains(s_mult_art, na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)

In [47]:
# calculate chunck size 
chunk_size = int(mult_art.shape[0]/NUM_CORE)

# split data into chunks 
chunks = [mult_art.iloc[mult_art.index[i:i + chunk_size]] for 
          i in range(0, mult_art.shape[0], chunk_size)]

In [48]:
# split up articles into smaller articles and append the resulting new articles 
# to the corpus
from datetime import datetime
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    results = pool.map(split_articles.split_articles, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

results = pd.concat(results)
print(len(results))
results.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

0:01:39.838263
899452


The separated articles consist of fewer words than the articles from which they originally stemmed. Therefore, we count the number of words of the new articles with the count_words_mp function from before and filter out articles with less than 100 words.

In [49]:
# count the number of words for the separated articles and filter out articles with less
# than 100 words
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in results['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)
results['word_count'] = count_results

0:00:04.964456


In [50]:
results['word_count'] = count_results
results = results[results['word_count']>=100]

In [51]:
# append separated articles to corpus
data = data.append(results)
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame
print(len(data))

3675441


In [52]:
#data.to_csv('dpa_prepro_step5.csv')

### dpa-Basisdienstes and dpa-afx Wirtschaftsnachrichten

Up to this point, we have been cleaning data from dpa-Basisdienstes (hereafter dpa) and dpa-afx Wirtschaftsnachrichten (hereafter afx) together. This made us realise that afx data has a completely different format and, while useful for forecasting, in our view should be analyzed separately from dpa data. Here we explain why.

The dpa news articles are very similar to the news we read in other newspapers, and they cover similar topics, namely major political and economic events.

The afx news, on the other hand, is aimed at investors and traders rather than the general public, and closely follows stock market developments, companies' financial results, and the state of key industries.

To make it clear what we mean, here are a few articles on topics that are frequently discussed in afx data.

1. The first two articles discuss stock market developments.
2. Articles three and four represent Ad hoc messages (information reported by companies that might influence the price of their securitites).
3. Articles five and six give recommendations to traders whether they should sell, buy, or hold shares.
4. The last two articles talk about financial results and plans of companies.

In [53]:
Stock_market = data[(data.texts.str.contains('^Aktien')) & (data.topic == 'afx')]
Ad_hoc = data[(data.texts.str.contains('Ad hoc-Service')) & (data.topic == 'afx')]
Analysis = data[(data.texts.str.contains('^AKTIE IM FOKUS')) & (data.topic == 'afx')]
Companies = data[(data.texts.str.contains('ROUNDUP')) & (data.topic == 'afx')]

print('---- Articles on the stock market ----')
print('\n')
print(Stock_market.reset_index()['texts'][0][:1000])
print('\n')
print(Stock_market.reset_index()['texts'][100][:1000])
print('\n')
print('---- Ad hoc messages ----')
print('\n')
print(Ad_hoc.reset_index()['texts'][0][:1000])
print('\n')
print(Ad_hoc.reset_index()['texts'][1][:1000])
print('\n')
print('---- Stock market analysis, recommendations for traders ----')
print('\n')
print(Analysis.reset_index()['texts'][0][:1000])
print('\n')
print(Analysis.reset_index()['texts'][1][:1000])
print('\n')
print('---- News on companies ----')
print('\n')
print(Companies.reset_index()['texts'][0][:1000])
print('\n')
print(Companies.reset_index()['texts'][10][:1000])
print('\n')

---- Articles on the stock market ----


Aktien Neuer Markt: Kurse abgebröckelt - Etwas Erleichterung durch US -Daten. FRANKFURT (dpa-AFX) - Die deutschen Wachstumsaktien am Neuen Markt sind nach einer freundlichen Eröffnung stetig abgebröckelt. Im Vorfeld der mit Spannung erwarteten US-Konjunkturzahlen am Nachmittag (ab 14:30) wurden neue Tagestiefs markiert, bevor die Daten «in line» für etwas Erleichterung sorgten. Der Nemax-All-Share-Index <NMDK.ETR> lag zuletzt um 1,08% unter dem Vortagesniveau bei 3.231,70/-35,13 Punkten, während der Nemax50-Index <NMKX.ETR> 1,23% auf 4.068,60/-50,94 Punkte verlor. Vor den Zahlen saßen «alle wie das Kaninchen vor der Schlange und warteten», erklärte ein Händler aus Düsseldorf. Offenbar rechneten einige Marktteilnehmer mit negativen Reaktionen der Märkte und gaben Stücke bei sehr geringer Handelstätigkeit aus der Hand. Dies habe den Gesamtmarkt im frühen Verlauf unter Druck gesetzt. Gleich nach den ersten Zahlen um 14:30 Uhr, bei denen die US-Verb

Overall, we can conclude that these types of articles are of interest to financial market participants, but may be too detailed for the general public.

A few exceptions, which we also consider, are afx articles on business cycle (Konjunktur) and Politics.

* Articles on business cycle might be very important for economic forecasting. We therefore considered including news reports with the keyword 'Konjunktur' (business cycle) published by afx. However, we encountered two problems.

* First, articles with the keyword 'Konjunktur' aimed at the general public often enter our database twice because they are published by both dpa and afx. Articles published exclusively by afx are often more quantitative and seem to target financial market participants. 

* Second, while the proportion of dpa articles with the keyword 'Konjunktur' is relatively stable and ranges from 0.5% to 4% (see 'dpa' column in the dataframe below), the proportion of 'Konjunktur' articles in afx data increases significantly from 3% in 2000 to 17% in 2018 (see 'afx' column). We believe this is due to the fact that in the afx data, the keyword 'Konjunktur' was initially used only in the articles with a lot of quantitative information, and then began to be used in a much wider range of articles. In any case, if we included all dpa articles and afx articles with the keyword 'Konjunktur', we would see that the share of articles with the keyword 'Konjunktur' increases over the years (see 'All' column), most likely due to structural changes in the database rather than economic developments. Therefore, we decided to exclude afx articles with the keyword 'Konjunktur' from further analysis.

* We also exclude afx articles with the keyword 'Politik', because in most cases dpa publishes the same articles. Thus, these articles can be considered as duplicates.

In [54]:
# The proportion of articles with the keyword 'Konjunktur' in the whole dataset.
Konjunktur_all_series = data[(data.keywords.str.contains('Konjunktur'))].groupby('year').nunique()['texts']/data[(data.topic == 'WiPo') | ((data.topic == 'afx') & (data.keywords.str.contains('Konjunktur')))].groupby('year').nunique()['texts']
Konjunktur_all_df = Konjunktur_all_series.to_frame().reset_index()
Konjunktur_all_df = Konjunktur_all_df.rename(columns = {"texts": "All"})
# The proportion of articles with the keyword 'Konjunktur' in dpa dataset.
Konjunktur_dpa_series = data[(data.keywords.str.contains('Konjunktur')) & (data.topic == 'WiPo')].groupby('year').nunique()['texts']/data[(data.topic == 'WiPo')].groupby('year').nunique()['texts']
Konjunktur_dpa_df = Konjunktur_dpa_series.to_frame().reset_index()
Konjunktur_dpa_df = Konjunktur_dpa_df.rename(columns = {"texts": "dpa"})
# The proportion of articles with the keyword 'Konjunktur' in afx dataset.
Konjunktur_afx_series = data[(data.keywords.str.contains('Konjunktur')) & (data.topic == 'afx')].groupby('year').nunique()['texts']/data[(data.topic == 'afx')].groupby('year').nunique()['texts']
Konjunktur_afx_df = Konjunktur_afx_series.to_frame().reset_index()
Konjunktur_afx_df = Konjunktur_afx_df.rename(columns = {"texts": "afx"})
# The dataframe with the above mentioned datafames merged together.
merged_df = Konjunktur_all_df.merge(Konjunktur_dpa_df, how = 'left', on = 'year')
merged_df = merged_df.merge(Konjunktur_afx_df, how = 'left', on = 'year')
merged_df

,year,All,dpa,afx
0,1991,0.005312,0.005312,NaN
1,1992,0.006777,0.006777,NaN
2,1993,0.008646,0.008646,NaN
3,1994,0.007072,0.007072,NaN
4,1995,0.006401,0.006401,NaN
5,1996,0.007545,0.007545,NaN
6,1997,0.007431,0.007431,NaN
7,1998,0.009868,0.009868,NaN
8,1999,0.007560,0.007560,NaN
9,2000,0.019270,0.008483,0.026158


All in all, we only keep articles published by dpa.

In [55]:
data = data[data.topic == 'WiPo']
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame
print(len(data))

2271472


In [56]:
data.to_csv('dpa_prepro_step6.csv')

In [57]:
data = pd.read_csv('dpa_prepro_step6.csv', encoding = 'utf-8', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'})

### Identify and Delete English Articles

In [37]:
# Delete all English articles from the data  
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    eng_results = pool.map(identify_eng.identify_eng, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

1:48:39.715753


In [48]:
data['language'] = eng_results
data = data[data.language==0]
data.reset_index(inplace=True, drop=True)
print(len(data))

2064333


In [ ]:
#data.to_csv('dpa_prepro_step7.csv')

### Identify articles that predominantly consist of numbers

Articles that consist predominately of numbers sometimes carry little sentiment. Filtering out all numbers only helps a little, because the resulting texts are often grammatical nonsensical. To get a better understanding of how these articles look we identify all articles that consist of more than 80% of numbers.

In [31]:
# use the 'numeric_articles' function to identify economic articles with a high share of numbers in them
inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.80))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    numeric_list = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:01:23.476436


In [32]:
numeric = data[numeric_list]

In [33]:
# inspect example article with high share of numbers
numeric['texts'].iloc[0]

'FLENSBURG (dpa-AFX) - Der langjährige Renner VW-Golf <VOW.ETR> ist auch 1999 an der Spitze der Auto-Neuzulassungen in Deutschland geblieben. Den 356.000 Golf-Exemplaren - einschließlich seiner Abwandlungen Vento und Bora - folgten erneut rund 230.000 Opel-Astra <GM.NYS>. Das geht aus einer Statistik der Kraftfahrt-Bundesamts (KBA) in Flensburg hervor, die am Dienstag veröffentlicht wurde. Insgesamt kamen 1999 knapp über 3,8 Millionen fabrikneue Personenwagen in den Straßenverkehr. 1998 waren es 3,73 Millionen.      Auf den dritten Platz liegen mit 143.000 Erstanmeldungen die Fahrzeugtypen der Dreier-Reihe von BMW <BME.ETR>, die den ebenfalls seit Jahren mit führenden VW-Passat (136.500) auf den vierten Rang verdrängten. Neuling in der 1999er Tabellenspitze der 25 Wagentypen - von insgesamt über 70 - sind der Ford-Focus <F.NYS> auf Platz fünf mit 113.000 Neuzulassungen, der VW-Lupo (16./61.000), der Peugeot 206 (22./44.000) sowie der Opel-Zafira (25./38.500). Auf Erfolgskurs ist auch d

In [34]:
numeric['texts'].iloc[1]

'Operative Kosten                        40.616     48.002    18,2% Vertriebs- und allg.                    25.892     36.359    40,4% Verwaltungskosten Zentrale Verwaltungskosten              16.114     18.884    17,2% Nicht liquiditätswirksame                    0      5.393 Personalkosten aus der Gewährung von Aktienoptionen Abschreibungen und Amortisierung       122.614    119.848    -2,3% ------------------------------------------------------------------ --- Betriebsaufwand                        205.236    228.486    11,3% ------------------------------------------------------------------ --- ------------------------------------------------------------------ --- Betriebsergebnis (-verlust)            -21.193    -21.268     0,4%  ------------------------------------------------------------------ --- Zinsaufwand                            -55.901    -31.046   -44,5% Anderer Aufwand                         -1.298     -1.500    15,6% --------------------------------------------------

In [35]:
len(numeric)

5412

In [ ]:
#data.to_csv('dpa_prepro_step8.csv')

### Merge continuations of articles

Some dpa-afx articles are split into multiple entries marked by the word 'Fortsetzung' at the beginning of the texts of following entries.

In [38]:
data['texts'].iloc[1]

'FRANKFURT (dpa-AFX) - Der deutsche Aktienmarkt hat am Montag für wenige Augenblicke seinen Rekordschluss vom Juli 1998 überschritten. Mit einem Höchststand von 6.188,68 lag der Dax über dem Tageschluss vom 21. Juli 1998, als der Index 6.186 Punkte erreichte. Der Dax <DAX.ETR> schloss den Handel am ersten Wochentag bei 6.142,19 Zählern und damit um 0,38% oder 23,02 Punkte fester ab. Der Nebenwerteindex M-Dax <MDAX.ETR> gab dagegen auf 3.990,00 Punkte oder um 1,43% nach, und der Neue Markt-Index Nemax 50 <NMKX.ETR> schloss bei 4.610,66 Zählern (-0,58%).      Spätestens auf dem Niveau des All-Time-High bei ungefähr 6.200 Punkten werde der Index auf einen massiven Widerstand stoßen, heißt es am Montag in einem Research-Report von der Nürnberger Schmidt Bank. Michael Schubert, Analyst der Bankgesellschaft Berlin, sagte in einem Gespräch mit dpa-AFX, derzeit sei der einzig negative Faktor, dass die Börse nicht von der Breite getragen werde, sondern nur von wenigen Titeln. Wer jetzt noch nic

In [39]:
data['texts'].iloc[3]

'(Fortsetzung) - Der Tagesgewinner SAP <SAP.ETR> übernahm bereits am Vormittag die Führung bei den Gewinnern ein und schloss den Handel mit einem Kurszuwachs von 6,27% auf 475,20 Euro ab. Händler waren sich einig darüber, dass SAP weiteres Wachstumpotenzial haben. Ende vergangener Woche habe es Kaufempfehlungen für das Papier gegeben, sagte ein Händler. Die ABN Amro Bank hatte ihr Kursziel mit 600 Euro festgesetzt.      Den Wert umgaben zudem Gerüchte über eine bevorstehende Allianz mit einem Weltkonzern, hieß es. Ein anderer Händler hielt den aktuellen SAP-Kurs noch immer für zu billig. Wachstumswerte stünden weiter in der Gunst der Anleger. Bei SAP hätten Image und Aktienkurs unter Problemen in den USA gelitten. Das werde nach Erwartung des Händlers aber spätestens mit den Geschäftszahlen des ersten Quartals 2000 überwunden sein. Mit dem Ende der Angst vor dem Jahr-2000-Problem würde der Kurs weiter steigen.     Ihren Kurshöhenflug fortsetzen konnten zudem die Aktien von Siemens <SIE

In [40]:
data['texts'].iloc[7]

'(Fortsetzung) - Überraschend fest tendierten am Montag die Autowerte. Volkswagen <VOW.ETR> gewannen 1,6% auf 49,35 Euro. Vielleicht sehen wir jetzt tatsächlich eine Branchen-Rotation zur Autobranche hin, sagte ein Händler der DG-Bank. Die nahe Zukunft des Wertes stehe oder falle mit der 50 Euro Marke. Auch BMW <BMW.ETR> (28,10 Euro/+1,04%) und DaimlerChrysler <DCX.ETR> (67,25 Euro/+0,67%)  verbuchten Kurszuwächse. Alle drei deutschen Hersteller verbuchten einem Pressebericht zufolge im Gesamtjahr 1999 steigende Absatzzahlen in den USA.       Die Verliererliste führten am Montag Lufthansa <LHA.ETR> (22,30 Euro/-3,88%) und Veba <VEB.ETR> (45,47 Euro/-3,87%) an. Unter Gewinnmitnahmen litt laut Händlern der Kurs der T-Aktie <DTE.ETR>: Der Titel verlor im Montagshandel 1,31% auf 59,30 Euro. Wäre die Telekom mit dem Indextrend im Wert gestiegen, hätte der Dax den Rekordstand vom Juli 1998 eingestellt, rechnete ein Händler am Nachmittag vor./mr/fs'

We identify which entries belong together and merge them to one article

In [46]:
# Divide data into roughly equal sized chunks where articles from one day only fall under the same chunk
data_cont = data[data['topic'] == 'afx']
dates = data_cont.groupby(['year', 'month', 'day'])
dates = list(dates.groups)
dates = np.array_split(dates, NUM_CORE)

meta_data = data_cont.loc[:, data_cont.columns != 'texts']

chunks = [pd.concat([data_cont[(data_cont['year'] == t[0]) & (data_cont['month'] == t[1]) & (data_cont['day'] == t[2])] for t in tup]) for tup in dates]
chunks = [[chunk, meta_data] for chunk in chunks]

In [48]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    continue_results = pool.map(continue_articles.continue_articles, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:14:56.636717


In [49]:
data.drop(list(itertools.chain(*[tup[0] for tup in continue_results])), inplace=True)

In [ ]:
continue_articles = pd.concat([tup[1] for tup in continue_results])
print(len(continue_articles))

In [50]:
data = data.append(continue_articles)
data.reset_index(inplace=True, drop=True)
print(len(data))

1974426


In [51]:
#data.to_csv('dpa_prepro_step9.csv')

## Remove URLs

In [52]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    url_corrected = pool.map(correct_url.correct_url, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:06:33.551105


In [53]:
data['texts'] = url_corrected

In [ ]:
#data.to_csv('dpa_prepro_step10.csv')

## Remove dpa references

We remove dpa references (e.g. NEW YORK (dpa) - ...) from each article.

In [54]:
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    dpa_ref_removed = pool.map(clean_dpa_references.clean_dpa_references, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:15:15.393896


In [55]:
data['texts'] = dpa_ref_removed

In [66]:
#data.to_csv('dpa_prepro_step11.csv')

# Correct spelling

### Umlauts

Older articles (1991 - 2000) from the section 'Kommentar' (Commentary) are often missing umlauts and correct capitalization. To
fix these two issues, we use the notebook Umlauts_fix written in Python 2 and the notebook Truecasing written in Python 3.

In [6]:
umlauts = ['ä', 'ö', 'ü', 'ß', 'Ä', 'Ö', 'Ü']
umlauts_replace = ['ae', 'oe', 'ue', 'ss', 'AE', 'OE', 'UE']

In [7]:
dpa_umlauts_fix = data[(data.texts.str.contains('|'.join(umlauts_replace))) & (~data.texts.str.contains('|'.join(umlauts))) & (data.year<2001)]

In [9]:
dpa_umlauts_fix['texts'].iloc[0]

'Ad-hoc announcement sent by DGAP. The sender is solely responsible for the contents of this announcement.  Ad-hoc Mitteilung Nach @ 15 WpHG  WizCom Technologies Ltd. (WizCom) <WZM.FSE> (Neuer Markt:WZM,WKN:915 856) veroeffentlicht das Ergebnis fuer das am 31. Dezember 1999 endende Geschaeftsjahr 20. Maerz 2000, Jerusalem, Israel - Der Umsatz belief sich im Jahr 1999 auf US$ Mio. 11,613. Der Vorjahreswert lag bei US$ Mio. 15,799. Der Umsatz im ersten bzw. zweiten Halbjahr 1999 betrug US$ Mio. 4,046 bzw. US$ Mio. 7,567. Das Unternehmen sieht den Umsatzrueckgang als voruebergehend an und vor allem im zweiten Halbjahr begruendet, weil das neue Produkt, der QuickLink-Pen, erst mit einigen Monaten Verzoegerung auf den Markt gebracht werden konnte. Im 4. Quartal 1999 konnte WizCom den QuickLink Pen in den USA erfolgreich auf den Markt bringen. Im 1. Quartal 2000 vermarktet WizCom den QuickLink Pen in weiteren Laendern, unter anderem Australien, Grossbritannien, Deutschland und Frankreich. De

In [ ]:
dpa_umlauts_fix.to_csv('dpa_umlauts_fix.csv', encoding='utf-8-sig', sep = ';')

In [ ]:
dpa_umlauts_fixed = pd.read_csv('dpa_umlauts_fixed.csv', encoding = 'utf-8', sep=';')

In [ ]:
#data.to_csv('dpa_prepro_step12.csv')

### Truecasing

In [56]:
dpa_cases_fix = data[data.texts.str.contains('^(?!.*[A-Z])')]

In [57]:
dpa_cases_fix['texts'].iloc[0]

'new york (vwd) - enttaeuschend verlief das geschaeft am mittwoch, dem ersten handelstag im neuen jahr, an der new yorker aktienboerse. die zunaechst gesehenen leichten gewinne konnten nur bis in das fruehe nachmittagsgeschaeft behauptet werden. in den letzten 2-1/2 geschaeftststunden gerieten die kurse in die minuszone und wall street schloss auf breiter front schwaecher. der dow-jones-index fuer 30 industriewerte gab um 23,02 auf 2.610,64 punkte nach. auch die uebrigen marktbestimmenden indizes gerieten in die minuszone. bei einem umsatz von 126,28 (114,13) millionen aktien standen die kursverlierer den -gewinnern im verhaeltnis von rund neun zu sieben gegenueber. verantwortlich fuer die schwaeche waren wiederauflebende befuerchtungen ueber eine anhaltende rezessionsphase. nachdem sogar das weisse haus jetzt von einer rezessionaeren entwicklung spricht, hielten sich die meisten anleger mit ihren engagements zurueck, wodurch der vorherrschende abgabedruck ausreichte, um die kurse in d

In [62]:
dpa_cases_fix.to_csv('dpa_case_fix.csv', encoding='utf-8-sig', sep = ';')

In [63]:
dpa_cases_fixed = pd.read_csv('dpa_cases_fixed.csv', encoding = 'utf-8', sep=';')

In [64]:
data.loc[dpa_cases_fixed.index, 'texts'] = dpa_cases_fixed

In [65]:
# fixed version
data['texts'].iloc[0]

'New York( Vwd)- Enttaeuschend verlief das Geschaeft am Mittwoch, dem ersten Handelstag im neuen Jahr, an der New Yorker Aktienboerse. Die Zunaechst Gesehenen leichten Gewinne konnten nur bis in das Fruehe Nachmittagsgeschaeft behauptet werden. In den letzten 2-1/2 Geschaeftststunden gerieten die Kurse in die Minuszone und Wall Street schloss auf breiter Front Schwaecher. Der Dow-Jones-Index Fuer 30 Industriewerte gab um 23,02 auf 2.610,64 Punkte nach. Auch die Uebrigen Marktbestimmenden Indizes gerieten in die Minuszone. Bei einem Umsatz von 126,28( 114,13) Millionen Aktien standen die Kursverlierer den -Gewinnern im Verhaeltnis von rund neun zu sieben Gegenueber. Verantwortlich Fuer die Schwaeche waren Wiederauflebende Befuerchtungen Ueber eine anhaltende Rezessionsphase. Nachdem sogar das Weisse Haus jetzt von einer Rezessionaeren Entwicklung spricht, hielten sich die meisten Anleger mit ihren Engagements Zurueck, wodurch der vorherrschende Abgabedruck ausreichte, um die Kurse in di

In [ ]:
#data.to_csv('dpa_prepro_step13.csv')

## Fixing tokens containing a number and a word

In quite a few cases, a number and a word are erroneously merged into a single token. Splitting these tokens into two tokens helps us to deal with the following problems:

(see Handelsblatt notebook)

In [58]:
startTime = datetime.now()

import split_number_word

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    split_corrected = pool.map(split_number_word.split_number_word, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:02:13.158789


In [59]:
data['texts'] = split_corrected

In [60]:
#data.to_csv('dpa_prepro_step14.csv')

# Delete Fuzzy Duplicates

In [61]:
types_keep = ['russische Aktienmarkt', 'europäischen Börsen', 'Deutsche Börse' ,'Europäische Zentralbank',
'Moskauer Aktienmarkt', 'deutsche Aktienmarkt', 'deutschen Aktienmarkt', 'Folgende Investmentbanken',
'deutsche Rentenmarkt', 'amerikanischen Treasury Bonds', 'IRW-PRESS', 'Deutsche Bank', 'Ausgewählte Analysten-Einstufungen',
'Deutsche Staatsanleihen', 'Der japanische Aktienmarkt']

exceptions = ['NO_EXCEPTIONS']

In [63]:
# import a function that outputs the indices of duplicates 
import fuzzy_duplicates
delete_indices = []
startTime = datetime.now() 
for year in list(set(data['year'])):
    data_input = data[(data['year'] == year)]
    for month in list(set(data_input[data_input['year'] == year]['month'])): # old: list(set(data))
            # Prepare inputs
            inputs_year = []
            inputs_month = []
            inputs_month_year = []
            inputs_year.append(year)
            inputs_month.append(month)
            inputs_month_year.append(data_input[(data_input['year'] == year) & (data_input['month'] == month)][["month", "year", "texts"]])

            #from itertools import repeat
            inputs = list(zip(inputs_year, inputs_month, inputs_month_year))
            from datetime import datetime
            if __name__ == "__main__":
                pool = mp.Pool(NUM_CORE)
                # apply function to all combinations of month-year in parallel
                delete_intermediate = pool.starmap(fuzzy_duplicates.fuzzy_duplicates, zip(inputs, repeat(types_keep), repeat(exceptions)))
                delete_indices = delete_indices + delete_intermediate # create one list of indices
                pool.close()
                pool.join()  
    print(year)
print(datetime.now()-startTime)

1991.0
1992.0
1993.0
1994.0
1995.0
1996.0
1997.0
1998.0
1999.0
2000.0
2001.0
2002.0
2003.0
2004.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0
2011.0
2012.0
2013.0
2014.0
2015.0
2016.0
2017.0
2018.0
6:58:11.847691


In [64]:
delete_indices = list(set([item for sublist in delete_indices for item in sublist]))

In [65]:
data.drop(delete_indices, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

1580324


In [66]:
#data.to_csv('dpa_prepro_step15.csv')

# Clean articles

Dpa articles include some unnecessary text passages like inquiry notes or references to webpages. We decided to clean the 
affected articles from these text passages to make the sentiment classification easier for our model(s).

We remove the following terms and sections from the texts:
* 1) stock symbols
* 2) additional metadata in the text meant for the author
* 3) references to previos articles
* 4) references to dpa and dpa-AFX webpage
* 5) uncorrected original article on which a correction is based on
* 6) inquiry notes
* 7) reference to english article on which some articles are based on
* 8) date of the article
* 9) references for aditional information (phone numbers, webpages etc.)
* 10) references to sender
* 11) references to Debitos
* 12) references to issuer
* 13) reference to authors
* 14) references to summary of article 

In [67]:
startTime = datetime.now()
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    cleaned_articles = pool.map(clean_dpa_articles.clean_dpa_articles, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:58:52.745164


In [68]:
data['texts'] = cleaned_articles

In [69]:
#data.to_csv('dpa_prepro_final.csv')